In [155]:
import time

import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [156]:
def parse_data(fpath):
    data = np.genfromtxt(fpath, dtype=np.int64, skip_header=7)
    X, y = data[:, :-1], data[:, -1]
    return X, y

In [197]:
fpath = "/Users/ChesterHuynh/classes/dldo467/dldo-hw/hw2/stable6.txt"

X, y = parse_data(fpath)
y -= 1  # Index classes starting at 0 for CrossEntropyLoss() to work

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [198]:
class My_Net(nn.Module): 
    def __init__(self, n_features, n_classes, n_neurons=10):
        super(My_Net, self).__init__()
        #feed forward layers
        self.layer_1 = nn.Linear(n_features, num_neurons)
        self.layer_2 = nn.Linear(num_neurons, num_neurons)
        self.layer_3 = nn.Linear(num_neurons, n_classes)

        #activations
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

In [217]:
n_classes = len(np.unique(y))
n_features = X_train.shape[1]

# Hyperparameters to tune
n_neurons = 20
lr = 1e-2

if "stable4" in fpath:
    batch_size = 1
elif "stable5" in fpath:
    batch_size = 32
elif "stable6" in fpath:
    batch_size = 64

In [ ]:
net=My_Net(n_features, n_classes, n_neurons=n_neurons)

# Standard cross entropy loss for multi-classification tasks
loss=nn.CrossEntropyLoss()

# Define the optimizer. Here we use Adam optimizer.
opt=torch.optim.Adam(net.parameters(), lr=lr)

In [218]:
Xtrain = torch.Tensor(X_train)
Xtest = torch.Tensor(X_test)
ytrain = torch.LongTensor(y_train)
ytest = torch.LongTensor(y_test)

In [219]:
train = torch.utils.data.TensorDataset(Xtrain, ytrain)
test = torch.utils.data.TensorDataset(Xtest, ytest)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)

In [220]:
#initialize the network using Xavier initialization.
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

In [221]:
net.apply(weights_init)

My_Net(
  (layer_1): Linear(in_features=15, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=20, bias=True)
  (layer_3): Linear(in_features=20, out_features=6, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [222]:
def train_eval(verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in train_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs,labels).item()
        num_batches += 1

    if verbose:
        print('Train accuracy: %f %%' % (100 * correct.item() / total))
    return loss_sum/num_batches, correct.item() / total

def test_eval(verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in test_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs,labels).item()
        num_batches += 1

    if verbose:
        print('Test accuracy: %f %%' % (100 * correct.item() / total))
    return loss_sum/num_batches, correct.item() / total

In [223]:
epochs = 25;
train_loss_store = []
train_acc_store = []
test_loss_store = []
test_acc_store = []

for epoch in range(epochs):
    time1 = time.time()
    print('In epoch %i : '%(epoch+1))
    for i, (feat, label) in enumerate(train_loader, 0):
        #set the gradients to zero initially for each batch
        opt.zero_grad()
        outputs = net(feat)
        l = loss(outputs, label)
        l.backward()
        opt.step()
    
    l_temp, acc_temp = train_eval()
    train_loss_store.append(l_temp)
    train_acc_store.append(acc_temp)

    l_temp, acc_temp = test_eval()
    test_loss_store.append(l_temp)
    test_acc_store.append(acc_temp)

    time2 = time.time()
    print("Time lapse: %f secs" %round((time2-time1), 2))

In epoch 1 : 
Train accuracy: 66.774208 %
Test accuracy: 66.330994 %
Time lapse: 0.990000 secs
In epoch 2 : 
Train accuracy: 67.083751 %
Test accuracy: 66.514088 %
Time lapse: 0.930000 secs
In epoch 3 : 
Train accuracy: 72.154161 %
Test accuracy: 71.539009 %
Time lapse: 0.940000 secs
In epoch 4 : 
Train accuracy: 73.741117 %
Test accuracy: 72.627403 %
Time lapse: 0.930000 secs
In epoch 5 : 
Train accuracy: 74.809260 %
Test accuracy: 73.705625 %
Time lapse: 0.980000 secs
In epoch 6 : 
Train accuracy: 74.879016 %
Test accuracy: 74.122673 %
Time lapse: 0.960000 secs
In epoch 7 : 
Train accuracy: 75.716092 %
Test accuracy: 74.712644 %
Time lapse: 0.950000 secs
In epoch 8 : 
Train accuracy: 75.389109 %
Test accuracy: 74.478690 %
Time lapse: 0.930000 secs
In epoch 9 : 
Train accuracy: 75.968958 %
Test accuracy: 75.068660 %
Time lapse: 0.970000 secs
In epoch 10 : 
Train accuracy: 76.356978 %
Test accuracy: 75.261927 %
Time lapse: 0.980000 secs
In epoch 11 : 
Train accuracy: 76.513929 %
Test a